# Image Compression

The goal of this notebook is to explore image compression using various forms of matrix decomposition.

In [1]:
import numpy as np
from numpy import random
import math
from matplotlib import pyplot as plt
from scipy import misc
import scipy
from tqdm import tqdm, trange
from time import perf_counter

In C:\Users\Sean\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Sean\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Sean\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Sean\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Sean\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle

In [2]:
%cd ..
from rputil import *
%cd -

C:\Users\Sean\random-projections
C:\Users\Sean\random-projections\notebooks


## ID2, RID2

In [3]:
import scipy.linalg.interpolative as sli

def id2_rank_k(A, k):
    idx, proj = sli.interp_decomp(A, k, rand=False)
    B = sli.reconstruct_skel_matrix(A, k, idx)
    C = sli.reconstruct_matrix_from_id(B, idx, proj)
    return C

def rid2_rank_k(A, k):
    idx, proj = sli.interp_decomp(A, k, rand=True)
    B = sli.reconstruct_skel_matrix(A, k, idx)
    C = sli.reconstruct_matrix_from_id(B, idx, proj)
    return C

In [4]:
def rid3_rank_k(A, k):
    cols = np.random.choice(A.shape[1], replace=False, size=k)
    AS = A[:,cols]
    Q, R = scipy.linalg.qr(AS, pivoting=False, mode='economic')
    return Q @ Q.T @ A

In [5]:
def transposerandomprojid(A, k): #A (m,n)
    Omega = np.random.randn(k, A.shape[0]) #(k,m)
    Y = (Omega @ A).T #(n,k)
    Q, R, P = scipy.linalg.qr(Y, mode='economic', pivoting=True)
    Q = Q.T
    return A @ Q.T @ Q

def normalrandomprojid(A, k):
    Omega = np.random.randn(A.shape[1], k) #(n,k)
    Y = A @ Omega # (m,k)
    Q, R, P = scipy.linalg.qr(Y, mode='economic', pivoting=True)
    return Q @ Q.T @ A

## Calculating Statistics

Here, we vary the rank k of the approximation and analyze the quality of the resulting compressed images:

In [6]:
import pandas as pd
mnist = pd.read_csv('../datasets/mnist/train.csv')

data = mnist[mnist.columns[1:]].values / 255
data = data[:10000]

In [7]:
plt.imshow(data[:5].reshape(28*5,28))

In [8]:
data = data.T

In [9]:
data_norm = np.linalg.norm(data)

In [10]:
def calculate_error(A):
    return np.linalg.norm(data - A) / data_norm

In [11]:
def stats(A):
    res = np.empty(7)
    
    res[0] = np.quantile(A, 0.25)
    res[1] = np.median(A)
    res[2] = np.quantile(A, 0.75)
    res[3] = np.mean(A)
    res[4] = np.std(A)
    res[5] = np.min(A)
    res[6] = np.max(A)
    
    return np.copy(res)

In [12]:
k_domain = np.concatenate(([1, 2, 5, 10], np.arange(20, 400, step=40), [400, 420, 440]))
k_len = k_domain.shape[0]

### deterministic methods

In [13]:
error_svd = np.empty(k_len)
error_id = np.empty(k_len)
error_id2 = np.empty(k_len)

time_svd = np.empty(k_len)
time_id = np.empty(k_len)
time_id2 = np.empty(k_len)

for i,val in enumerate(tqdm(k_domain)):
    #SVD
    start = perf_counter()
    error_svd[i] = calculate_error(svd_rank_k(data, val))
    time_svd[i] = perf_counter() - start
    
    #ID
    start = perf_counter()
    error_id[i] = calculate_error(id_rank_k(data, val))
    time_id[i] = perf_counter() - start
    
    #ID2 (SciPy)
    start = perf_counter()
    error_id2[i] = calculate_error(id2_rank_k(data, val))
    time_id2[i] = perf_counter() - start

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [04:22<00:00, 15.43s/it]


### randomized methods

In [17]:
stat_dims = (7, k_len)
error_random_svd = np.empty(stat_dims)
error_random_id = np.empty(stat_dims)
error_random_id2 = np.empty(stat_dims)
error_random_id3 = np.empty(stat_dims)
error_trpid = np.empty(stat_dims)
error_nrpid = np.empty(stat_dims)

time_random_svd = np.empty(k_len)
time_random_id = np.empty(k_len)
time_random_id2 = np.empty(k_len)
time_random_id3 = np.empty(k_len)
time_trpid = np.empty(k_len)
time_nrpid = np.empty(k_len)


iterations = 3 #10

for i,val in enumerate(tqdm(k_domain)):
    err_rsvd = np.empty(iterations)
    err_rid = np.empty(iterations)
    err_rid2 = np.empty(iterations)
    err_rid3 = np.empty(iterations)
    err_trpid = np.empty(iterations)
    err_nrpid = np.empty(iterations)
    
    t_rsvd = np.empty(iterations)
    t_rid = np.empty(iterations)
    t_rid2 = np.empty(iterations)
    t_rid3 = np.empty(iterations)
    t_trpid = np.empty(iterations)
    t_nrpid = np.empty(iterations)
    
    for j in range(iterations):
        #RSVD
        start = perf_counter()
        err_rsvd[j] = calculate_error(random_svd_rank_k(data,val))
        t_rsvd[j] = perf_counter() - start
        
        #RID
        start = perf_counter()
        err_rid[j] = calculate_error(random_id_rank_k(data,val))
        t_rid[j] = perf_counter() - start
        
        #RID2 (SciPy)
        start = perf_counter()
        err_rid2[j] = calculate_error(rid2_rank_k(data,val))
        t_rid2[j] = perf_counter() - start
        
        #RID3 (no oversampling + no pivoting)
        start = perf_counter()
        err_rid3[j] = calculate_error(rid3_rank_k(data,val))
        t_rid3[j] = perf_counter() - start
        
        #Tranpose RP ID
        start = perf_counter()
        err_trpid[j] = calculate_error(transposerandomprojid(data,val))
        t_trpid[j] = perf_counter() - start
        
        #Normal RP ID
        start = perf_counter()
        err_nrpid[j] = calculate_error(normalrandomprojid(data, val))
        t_nrpid[j] = perf_counter() - start
    
    error_random_svd[:,i] = stats(err_rsvd)
    error_random_id[:,i] = stats(err_rid)
    error_random_id2[:,i] = stats(err_rid2)
    error_random_id3[:,i] = stats(err_rid3)
    error_trpid[:,i] = stats(err_trpid)
    error_nrpid[:,i] = stats(err_nrpid)
    
    time_random_svd[i] = np.mean(t_rsvd)
    time_random_id[i] = np.mean(t_rid)
    time_random_id2[i] = np.mean(t_rid2)
    time_random_id3[i] = np.mean(t_rid3)
    time_trpid[i] = np.mean(t_trpid)
    time_nrpid[i] = np.mean(t_nrpid)

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [07:58<00:00, 28.12s/it]


## Display results

In [21]:
fig, ax = plt.subplots(1,2, figsize=(20,10))

ax[0].plot(k_domain, error_svd, c='r', label='SVD')
ax[0].plot(k_domain, error_id, c='g', label='ID')
ax[0].plot(k_domain, error_id2, c='b', label='scipy ID')
ax[0].plot(k_domain, error_random_svd[3], c='m', label='mean RSVD')
ax[0].plot(k_domain, error_random_id[3], c='y', label='mean RID')
ax[0].plot(k_domain, error_random_id2[3], c='c', label='mean scipy RID')
ax[0].plot(k_domain, error_random_id3[3], c='black', label='mean RID (no o+p)')
ax[0].plot(k_domain, error_trpid[3], c='#aabbcc', label='mean transpose RPID')
ax[0].plot(k_domain, error_nrpid[3], c='#ccbbaa', label='mean normal RPID')
ax[0].set_xlabel('k')
ax[0].set_ylabel('Relative Error')
# ax[0].set_yscale('log')
ax[0].legend(loc='upper right')
ax[0].set_title('Matrix Approximations: Error vs. Rank')

ax[1].plot(k_domain, time_svd, c='r', label='SVD')
ax[1].plot(k_domain, time_id, c='g', label='ID')
ax[1].plot(k_domain, time_id2, c='b', label='scipy ID')
ax[1].plot(k_domain, time_random_svd, c='m', label='mean RSVD')
ax[1].plot(k_domain, time_random_id, c='y', label='mean RID')
ax[1].plot(k_domain, time_random_id2, c='c', label='mean scipy RID')
ax[1].plot(k_domain, time_random_id3, c='black', label='mean RID (no o+p)')
ax[1].plot(k_domain, time_trpid, c='#aabbcc', label='mean transpose RPID')
ax[1].plot(k_domain, time_nrpid, c='#ffbbaa', label='mean normal RPID')
ax[1].set_xlabel('k')
ax[1].set_ylabel('Time')
ax[1].set_yscale('log')
ax[1].legend(loc='upper right')
ax[1].set_title('Matrix Approximations: Time vs. Rank');

In [23]:
fig, ax = plt.subplots(1, figsize=(5,5))


ax.fill_between(k_domain, error_random_svd[5], error_random_svd[6], color='m')
ax.fill_between(k_domain, error_random_id[5], error_random_id[6], color='y')
ax.fill_between(k_domain, error_random_id2[5], error_random_id2[6], color='c')
ax.fill_between(k_domain, error_random_id3[5], error_random_id3[6], color='black')
ax.fill_between(k_domain, error_trpid[5], error_trpid[6], color='#aabbcc')
ax.fill_between(k_domain, error_nrpid[5], error_nrpid[6], color='#ffbbaa')

ax.fill_between(k_domain, error_random_svd[0], error_random_svd[2], color='r')
ax.fill_between(k_domain, error_random_id[0], error_random_id[2], color='g')
ax.fill_between(k_domain, error_random_id2[0], error_random_id2[2], color='b')
ax.fill_between(k_domain, error_random_id3[0], error_random_id3[2], color='grey')
ax.fill_between(k_domain, error_trpid[0], error_trpid[2], color='#aabbcc')
ax.fill_between(k_domain, error_nrpid[0], error_nrpid[2], color='#ffbbaa')


ax.plot(k_domain, error_random_svd[1], c='m', label='RSVD')
ax.plot(k_domain, error_random_id[1], c='y', label='RID')
ax.plot(k_domain, error_random_id2[1], c='c', label='RID2')
ax.plot(k_domain, error_random_id3[1], c='black', label='RID3')
ax.plot(k_domain, error_trpid[1], c='#aabbcc', label='Tranpose RPID')
ax.plot(k_domain, error_nrpid[1], c='#ffbbaa', label='Normal RPID')
ax.set_xlabel('k')
ax.set_ylabel('Relative Error')
ax.legend(loc='upper right')
ax.set_title('Error IQR');

In [24]:
def temp_rid(A, k):
    AS = A[:,:k]
    Q, R = scipy.linalg.qr(AS, pivoting=False, mode='economic')
    return Q @ Q.T @ A

In [25]:
mat = np.copy(data)

In [26]:
temp_k = 200

In [27]:
approx = temp_rid(mat, temp_k)

In [28]:
eps = 0.1

print(f'{np.sum(approx > eps) / approx.size:.1%} of entries are greater than {eps}')

22.5% of entries are greater than 0.1


In [29]:
print(f'The average value of an entry is {np.mean(approx):.4f}')

The average value of an entry is 0.1313


In [30]:
mat.shape

(784, 10000)

In [31]:
approx.shape

(784, 10000)

In [32]:
total = 0
for i in trange(approx.shape[1]):
    col = approx[:,i]
    for j in range(temp_k):
        if (col - mat[:,j] < 0.001).all():
            total += 1

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:33<00:00, 294.70it/s]


In [33]:
print(total)

200
